In [1]:
from Model import Net 
import os
from dataLoader import yield_train_data 
import torch.nn.functional as F 
import torch 
from torch.autograd import Variable
from torch_geometric.data import Data
import numpy as np

# 引入attribute向量
attribute_embedding = np.load("generateData/Attribute_High_Embedding.npy") 
shop_size = attribute_embedding.shape[0]+1    #pad 0
# 引入category向量
category_embedding = np.load("generateData/Category_Embedding.npy")
# shop特征矩阵
shop_matrix = np.concatenate((attribute_embedding, category_embedding), axis=1)
padLine = np.zeros([1, attribute_embedding.shape[1]+category_embedding.shape[1]])
shop_matrix = np.insert(shop_matrix, 0, values=padLine, axis=0)      # pad 0
shop_size = shop_matrix.shape[0]


struct_matrix = np.load("generateData/Spatial_Graph_200m.npy")
edge_index = torch.tensor(struct_matrix, dtype=torch.long).cuda()
data = Data(x=shop_matrix, edge_index=edge_index)

BATCH_SIZE = 256
model = Net(shop_matrix) 
lr = 0.0005
epoches = 20 
weight_decay = 1e-5 
loss_score = 10000

save_path = "MyNetALL.pkl"
if save_path and not os.path.exists(save_path):
    f = open(save_path, "w")
    f.close()

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay) 
if torch.cuda.is_available(): 
    model.cuda()

# 查看网络中要训练的参数，网络中所有的参数结构
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

model.train()
for epoch in range(epoches): 
    running_loss = 0.0 
    batch_round = 0
    for train_data in yield_train_data(500*BATCH_SIZE): 
        userIndex = train_data["userIndex"]
        userHistoryIndex = train_data["userHistoryIndex"] 
        poiIndex = train_data["poiIndex"]
        labelIndex = train_data["labelIndex"]

        user = np.array(userIndex, dtype=np.int32)
        poi = np.array(poiIndex, dtype=np.int32)
        label = np.array(labelIndex, dtype=np.int32)

        shop_idxs = Variable(torch.LongTensor(list(range(shop_size))))       # pad 0

        batch_iter = user.shape[0] // BATCH_SIZE
        for i in range(batch_iter+1):
            offset = i * BATCH_SIZE
            end = min(offset+BATCH_SIZE, user.shape[0])
            if offset == end:
                break

            batch_user = Variable(torch.LongTensor(user[offset:end]))

            batch_userHistoryList = userHistoryIndex[offset:end]
            batch_userHistoryLength = []
            for idx, element in enumerate(batch_userHistoryList):
                batch_userHistoryLength.append(len(element))
            batch_userHistoryLength = Variable(torch.LongTensor(batch_userHistoryLength))

            max_len = max(len(element) for idx, element in enumerate(batch_userHistoryList))
            userHistory = np.zeros([len(batch_userHistoryList), max_len], dtype=np.int32)
            for iIndex, element in enumerate(batch_userHistoryList):
                for jIndex, sub_element in enumerate(element):
                    userHistory[iIndex][jIndex] = sub_element
            batch_userHistory = Variable(torch.LongTensor(userHistory))
            batch_poi = Variable(torch.LongTensor(poi[offset:end]))
            batch_label = Variable(torch.LongTensor(label[offset:end]))

            if torch.cuda.is_available():
                shop_idxs, batch_user, batch_userHistory, batch_userHistoryLength, batch_poi, batch_label = shop_idxs.cuda(), batch_user.cuda(), batch_userHistory.cuda(), batch_userHistoryLength.cuda(), batch_poi.cuda(), batch_label.cuda()

            optimizer.zero_grad()
            outputs = model(shop_idxs, data, batch_user, batch_poi, batch_userHistory, batch_userHistoryLength)
            outputs = F.log_softmax(outputs, dim=1)
            loss = F.nll_loss(outputs, batch_label, reduction="sum")
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i%10 == 9:
                batch_round += 1
                print('Train Epoch: %d, Batch %d, loss:%.6f' % (epoch+1, batch_round, running_loss/(10*BATCH_SIZE)))
                if(running_loss<loss_score):
                    loss_score = running_loss
                    torch.save(model.state_dict(), save_path)
                    print("*************************Save model successful********************************")
                running_loss = 0
            model.train()
print("Finished Training")

Model's state_dict:
w_omega 	 torch.Size([32, 32])
u_omega 	 torch.Size([32, 1])
w_omega2 	 torch.Size([32, 32])
u_omega2 	 torch.Size([32, 1])
w_omega3 	 torch.Size([32, 32])
u_omega3 	 torch.Size([32, 1])
shop_embedding.weight 	 torch.Size([21041, 32])
conv1.lin.weight 	 torch.Size([32, 32])
conv1.lin.bias 	 torch.Size([32])
bn.weight 	 torch.Size([32])
bn.bias 	 torch.Size([32])
bn.running_mean 	 torch.Size([32])
bn.running_var 	 torch.Size([32])
bn.num_batches_tracked 	 torch.Size([])
conv2.lin.weight 	 torch.Size([32, 32])
conv2.lin.bias 	 torch.Size([32])
lstm1.weight_ih_l0 	 torch.Size([128, 32])
lstm1.weight_hh_l0 	 torch.Size([128, 32])
lstm1.bias_ih_l0 	 torch.Size([128])
lstm1.bias_hh_l0 	 torch.Size([128])
lstm2.weight_ih_l0 	 torch.Size([128, 32])
lstm2.weight_hh_l0 	 torch.Size([128, 32])
lstm2.bias_ih_l0 	 torch.Size([128])
lstm2.bias_hh_l0 	 torch.Size([128])
fc1.0.weight 	 torch.Size([64])
fc1.0.bias 	 torch.Size([64])
fc1.0.running_mean 	 torch.Size([64])
fc1.0.runnin

KeyboardInterrupt: 